In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time

# 1st link

In [2]:
def mineralsExtract(mineralMainCategory, mineralsList, minerals, parentList):
    dfnative = pd.DataFrame()
    for subCategory in mineralsList:
        for subelements in subCategory.find_all("dd")[:2]:
            if subelements.find("a").text not in minerals:
                minerals.append(subelements.find("a").text)
                parentList.append(mineralMainCategory)
    return [minerals, parentList]

In [3]:
def scrape_pages(URL):
    max_pages = 9
    current_page = 1
    minerals = []
    parent = []
    finalDF = pd.DataFrame()
    while current_page <= max_pages+1:
        print(f'{URL}{current_page}')
        if current_page == 10:
            content = requests.get(f'{URL[:-1]}{current_page}').text
        else:
            content = requests.get(f'{URL}{current_page}').text
        mineralSoup = BeautifulSoup(content, 'html.parser')
        mineralDataClass1 = mineralSoup.find("blockquote")
        mineralMainCategory = mineralDataClass1.find("h2").text
        result = re.search("-(.*)S ", mineralMainCategory)
        mineralMainCategory = str(str(result.group(1)) + "S").lstrip().replace(",","")#print(result.group(1))
        print(mineralMainCategory)
        minerals, parent = mineralsExtract(mineralMainCategory, mineralDataClass1.find_all("blockquote", limit=5), minerals, 
                                           parent)
        time.sleep(15) 
        current_page += 1
    finalDF = pd.DataFrame({"Mineral":minerals, "Parent": parent, "Relation": "instanceOf"})
    finalDF.to_excel("outputmineralnew.xlsx")
    finalDF.to_csv("outputmineralnew.csv")
    return finalDF


def main():
    URL = 'https://www.webmineral.com/strunz/strunz.php?class=0'
    data = scrape_pages(URL)

if __name__ == '__main__':
    exit(main())

https://www.webmineral.com/strunz/strunz.php?class=01
ELEMENTS
https://www.webmineral.com/strunz/strunz.php?class=02
SULFIDES and SULFOSALTS
https://www.webmineral.com/strunz/strunz.php?class=03
HALIDES
https://www.webmineral.com/strunz/strunz.php?class=04
OXIDES
https://www.webmineral.com/strunz/strunz.php?class=05
CARBONATES
https://www.webmineral.com/strunz/strunz.php?class=06
BORATES
https://www.webmineral.com/strunz/strunz.php?class=07
SULFATES
https://www.webmineral.com/strunz/strunz.php?class=08
PHOSPHATES ARSENATES VANADATES
https://www.webmineral.com/strunz/strunz.php?class=09
SILICATES
https://www.webmineral.com/strunz/strunz.php?class=010
ORGANIC COMPOUNDS


# 2nd link

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

html_content = requests.get('https://www.nmnhs.com/gemmology/classification/list_minerals-en.html', headers=headers).text
soup = BeautifulSoup(html_content, 'html.parser')

In [5]:
mineral2 = soup.find("div", class_="box_emph_y")

In [6]:
mineral = []
parents = []
finalDF2 = pd.DataFrame()
for i, header in enumerate(mineral2.find_all('b'), 1):
    next_tag = header
    j = 1
    while True:
        next_tag = next_tag.next_sibling
        if next_tag is None or next_tag.name == 'b':
            break
        if next_tag.name is not None:
            parent = header.text.split(" (")[0].upper().replace("&", "and").replace(",", "")
            parent = "OXIDES" if (parent == "OXIDES and HYDROXIDES") else parent
            parent = "SULFATES" if (parent == "SULPHATES") else parent
            parent = "SULFIDES and SULFOSALTS" if (parent == "SULPHIDES and SULPHOSALTS") else parent
            if(next_tag.text and (next_tag.has_attr('class') and next_tag['class'][0] == 'n_body_blt')):
                result = re.search("[0-9].(.*) ", next_tag.text)
                mineral_name = next_tag.text.split(result.group(0))[1].replace(" ", "_")
                mineral.append(mineral_name)
                parents.append(parent)
            j += 1
        finalDF2 = pd.DataFrame({"Mineral":mineral, "Parent": parents, "Relation": "instanceOf"})
print(finalDF2)

         Mineral                   Parent    Relation
0        Diamond                 ELEMENTS  instanceOf
1     Sphalerite  SULFIDES and SULFOSALTS  instanceOf
2       Fluorite                  HALIDES  instanceOf
3    Chrysoberyl                   OXIDES  instanceOf
4         Spinel                   OXIDES  instanceOf
..           ...                      ...         ...
101    Marialite                SILICATES  instanceOf
102    Natrolite                SILICATES  instanceOf
103    Scolecite                SILICATES  instanceOf
104      Leucite                SILICATES  instanceOf
105    Pollucite                SILICATES  instanceOf

[106 rows x 3 columns]


# Data Aggregation

In [7]:
final = finalDF2.to_csv('outputmineralnew.csv', mode='a', header=False)

In [ ]:
data = pd.read_csv("outputmineralnew.csv")
data = data.drop_duplicates(['Mineral'])
data.to_csv("outputmineralremoveduplicates.csv")

# Save to existing ontology

In [ ]:
#loading the ontology:
import pandas as pd
from owlready2 import *
onto = get_ontology("file://C:/Users/Lenovo/Desktop/UNI-Koblenz-Landua/Research/Coding/geological-taxonomy.owl").load()

In [ ]:
data = pd.read_csv("outputmineralremoveduplicates.csv")

In [ ]:
for index, row in data.iterrows():
    with onto:
        mineral = row['Mineral']
        parent = row['Parent'].lower().replace(" ","_")
        for mineralCat in list(onto.Mineral.subclasses()):
            if(parent == mineralCat.name.lower()):
                mineral_name = mineralCat(mineral)

In [ ]:
for i in onto.Sulfates.instances(): print(i)

In [ ]:
onto.save(file = "../geological-taxonomy.owl", format = "rdfxml")